In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

def simulation_data(bounds):
    # define inputs to the simulation - data to be analysed
    mass = [random.uniform(200, 400) for _ in range(3)]
    CG = [random.uniform(2.5, 3.5) for _ in range(3)]
    input = np.array([(m, c) for m in mass for c in CG]).reshape((9, 2))
    #print("grid(mass, CG): ", input)
    
    # add automation to simulate data
    
    # Obtain outputs from the simulation - chosen initial solution population
    load_factor = np.array([1, 2, 3, 4, 5, 4, 3, 2, 1])
    velocity = np.array([300, 400, 500, 600, 700, 600, 300, 200, 100])
    output = np.concatenate((load_factor.reshape(-1, 1), velocity.reshape(-1, 1)), axis=1)
    #print("simulated output(Load_factor,velocity): ", output)
    return input,output

# define Overall simulated data function
def overall_data(input,output):
    sim_data = {}
    for i in range(len(input)):
        sim_data[str((input[i][0], input[i][1]))] = str((output[i][0], output[i][1]))
    #print("Simulated data(mass,CG = load_factor,velocity):",sim_data)
    return sim_data

# define Input separation function
def input_data(sim_data):
    # input variables
    mass = [eval(k)[0] for k in sim_data.keys()]
    CG = [eval(k)[1] for k in sim_data.keys()]
    load_factor = [eval(i)[0] for i in sim_data.values()]
    velocity = [eval(i)[1] for i in sim_data.values()] 
    return mass,CG,load_factor,velocity

# define Objective function
def objective_function( mass, CG, load_factor, velocity):
    #failure criteria variables
    load_factor_limit = 5
    velocity_limit = 700
    #failure objective function
    fail_region = (lf - load_factor) + (velocity - velocity_limit)
    return fail_region

# define generate_population function
def generate_population(n_pop, n_bits_total, pop_bounds):
    # repeat the bounds for each candidate solution
    pop_bounds = np.tile(pop_bounds, (n_pop, 1))
    # generate random bitstrings
    pop = np.random.rand(n_pop, n_bits_total)
    # convert bitstrings to decision variables
    pop = pop_bounds[:, 0] + pop * (pop_bounds[:, 1] - pop_bounds[:, 0])
    return pop

# define the selection function
def selection(pop, scores, k):
    # first random selection
    selection_ix = np.random.randint(0, len(pop))
    print(" initial selection: ", selection_ix)
    for ix in [np.random.randint(0, len(pop)) for _ in range(k-1)]:
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
            print("selection", selection_ix)
    return pop[selection_ix]

# define the crossover function
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if np.random.rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = np.random.randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

# define the mutation function
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if np.random.rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]
            
#define update_population
def update_population():
    return 

# define the genetic algorithm
def genetic_algorithm(bounds, n_bits, n_pop, n_gen, n_sel, r_cross, r_mut, sim_data):
    # initial population of random bitstrings
    pop = [np.random.randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = 0, np.inf
    # enumerate generations
    for gen in range(n_gen):
        # decode population
        decoded = [(bounds[i][0] + sum([pop[j][i]*2**(-1*(k+1)) for k in range(n_bits)])*(bounds[i][1] - bounds[i][0])/(1-2**(-1*n_bits))) for j in range(n_pop) for i in range(len(bounds))]
        # evaluate all candidates in the population
        scores = [objective_function(weights=decoded[i:i+4], sim_data=sim_data) for i in range(0, len(decoded), 4)]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = decoded[i:i+4], scores[i]
                print(">%d, new best f(%s) = %.3f" % (gen,  best, best_eval))
        # select parents
        selected = [selection(pop, scores,k) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get pairs of parents
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]

#____________________MAIN FUNCTION_____________________________________

# Grid boundaries for inputs
bounds = [[0, 1], [0, 1]]

# define the total iterations
n_iter = 100
# bits per variable
n_bits = 16
# define the population size
n_pop = 50
# define the number of generations
n_gen = 100
# selection number 
n_sel = 6
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * 2.0)

# define the simulation data processing
input,output = simulation_data(bounds) 

# make a dictionary of the simulated data
sim_data = overall_data(input,output)
 
# perform the genetic algorithm search
best, best_eval = genetic_algorithm(bounds, n_bits, n_pop, n_gen, n_sel, r_cross, r_mut, sim_data) 

# write the output to a file
with open('output.txt', 'w') as f:
    f.write(f'Best solution: {best}\\nScore: {best_eval}\\n')
    
print('Done!')